In [1]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

In [2]:
# read function
def readCsv(name):
    try:
        return pd.read_csv(name)
    except UnicodeDecodeError:
        try:
            return pd.read_csv(name, encoding='latin1')
        except UnicodeDecodeError:
            return pd.read_csv(name, encoding='ISO-8859-1')

In [3]:
apptsDf = readCsv('csv/appointments.csv')
apptsDf.drop_duplicates(subset='apptid', inplace=True)

In [4]:
clinicsDf = readCsv('csv/clinics.csv')
clinicsDf.drop_duplicates(subset='clinicid', inplace=True)

In [5]:
doctorsDf = readCsv('csv/doctors.csv')
doctorsDf.drop_duplicates(subset='doctorid', inplace=True)

In [6]:
pxDf = readCsv('csv/px.csv')
pxDf.drop_duplicates(subset='pxid', inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_8620\3144380163.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(name)


In [7]:
apptsDf = apptsDf[apptsDf["clinicid"].isin(clinicsDf["clinicid"])]
apptsDf = apptsDf[apptsDf["doctorid"].isin(doctorsDf["doctorid"])]
apptsDf = apptsDf[apptsDf["pxid"].isin(pxDf["pxid"])]


In [8]:
clinicsDf = clinicsDf[clinicsDf["clinicid"].isin(apptsDf["clinicid"])]
doctorsDf = doctorsDf[doctorsDf["doctorid"].isin(apptsDf["doctorid"])]
pxDf = pxDf[pxDf["pxid"].isin(apptsDf["pxid"])]


In [ ]:
pxDf = pxDf.rename(columns={'age': 'pxage', 'gender': 'pxgender'})
doctorsDf = doctorsDf.rename(columns={'age': 'doctorage'})

In [ ]:
combinedDf = apptsDf.merge(clinicsDf, on='clinicid')
combinedDf = combinedDf.merge(doctorsDf, on='doctorid')
combinedDf = combinedDf.merge(pxDf, on='pxid')

In [ ]:
combinedDf.drop(['pxid', 'clinicid', 'doctorid'], axis=1, inplace=True)


In [ ]:
Luzon = 'National Capital Region (NCR)', 'CALABARZON (IV-A)', 'Ilocos Region (I)', 'Bicol Region (V)','Central Luzon (III)'
luzonDf = combinedDf[combinedDf['RegionName'].isin(Luzon)]
visMinDf = combinedDf[~combinedDf['RegionName'].isin(Luzon)]

In [ ]:
def table(cnx):
    cursor = cnx.cursor()
    cursor.execute("CREATE DATABASE IF NOT EXISTS distributed_database")
    cursor.execute("USE distributed_database")
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS appointments (
            `apptid` VARCHAR(50) NOT NULL,
            `status` VARCHAR(50) NULL,
            `TimeQueued` DATETIME(1) NULL,
            `QueueDate` DATETIME(1) NULL,
            `StartTime` DATETIME(1) NULL,
            `EndTime` DATETIME(1) NULL,
            `type` VARCHAR(45) NULL,
            `Virtual` BOOL NULL,
            `hospitalname` VARCHAR(255) NULL,
            `IsHospital` BOOL NULL,
            `City` VARCHAR(255) NULL,
            `Province` VARCHAR(255) NULL,
            `RegionName` VARCHAR(255) NULL,
            `mainspecialty` VARCHAR(255) NULL,
            `doctorage` INT NULL,
            `pxage` INT NULL,
            `pxgender` VARCHAR(50) NULL,
            PRIMARY KEY (`apptid`)
        )
    """)
    cnx.commit()


In [ ]:
cnx = mysql.connector.connect(
    host='ccscloud.dlsu.edu.ph',
    port=20141,
    user='root',
)
table(cnx)
cnx.close()

In [ ]:
engine = create_engine('mysql+pymysql://root@ccscloud.dlsu.edu.ph:20141/distributed_database')
combinedDf.to_sql('appointments', con=engine, if_exists='replace', index=False)
engine.dispose()


In [ ]:
cnx = mysql.connector.connect(
    host='ccscloud.dlsu.edu.ph',
    port=20142,
    user='root',
)
table(cnx)
cnx.close()

In [ ]:
engine = create_engine('mysql+pymysql://root@ccscloud.dlsu.edu.ph:20142/distributed_database')
luzonDf.to_sql('appointments', con=engine, if_exists='replace', index=False)
engine.dispose()


In [ ]:
cnx = mysql.connector.connect(
    host='ccscloud.dlsu.edu.ph',
    port=20143,
    user='root',
)
table(cnx)
cnx.close()

In [ ]:
engine = create_engine('mysql+pymysql://root@ccscloud.dlsu.edu.ph:20143/distributed_database')
visMinDf.to_sql('appointments', con=engine, if_exists='replace', index=False)
engine.dispose()